## Install Dependencies

In [6]:
# %pip install earthengine-api -q
# %pip install git+https://github.com/cordmaur/GEES2Downloader.git@main -q
# %pip install geemap -q

In [7]:
# %pip install pympler

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import ee
import matplotlib.pyplot as plt
from pathlib import Path

import xarray as xr
import rasterio as rio

from waterdetect.engine import search_tiles, WaterDetect
from waterdetect.cloudless import get_gee_img

In [10]:
# ee.Authenticate()
ee.Initialize()

In [11]:
# For memory tracking

# %pip install pympler

from IPython.display import clear_output

# # from pympler.tracker import SummaryTracker
# from pympler.classtracker import ClassTracker
# # tracker = SummaryTracker()
# tr = ClassTracker()
# tr.track_class(WaterDetect)

# tracker.print_diff()

# Getting the Images to be Processed

In [12]:
tiles = search_tiles('23KLT', '2021-01/2021-12')
tiles

[<Item id=S2A_MSIL2A_20211230T132241_R038_T23KLT_20211231T153016>,
 <Item id=S2A_MSIL2A_20211227T131241_R138_T23KLT_20211228T010021>,
 <Item id=S2B_MSIL2A_20211225T132229_R038_T23KLT_20211226T003424>,
 <Item id=S2B_MSIL2A_20211222T131239_R138_T23KLT_20211223T144408>,
 <Item id=S2A_MSIL2A_20211220T132231_R038_T23KLT_20211221T133024>,
 <Item id=S2A_MSIL2A_20211217T131241_R138_T23KLT_20211218T021014>,
 <Item id=S2B_MSIL2A_20211215T132229_R038_T23KLT_20211216T075359>,
 <Item id=S2B_MSIL2A_20211212T131239_R138_T23KLT_20211213T071006>,
 <Item id=S2A_MSIL2A_20211210T132231_R038_T23KLT_20211210T220659>,
 <Item id=S2A_MSIL2A_20211207T131241_R138_T23KLT_20211208T090357>,
 <Item id=S2B_MSIL2A_20211205T132229_R038_T23KLT_20211205T214751>,
 <Item id=S2B_MSIL2A_20211202T131239_R138_T23KLT_20211203T024635>,
 <Item id=S2A_MSIL2A_20211130T132231_R038_T23KLT_20211201T040048>,
 <Item id=S2A_MSIL2A_20211127T131241_R138_T23KLT_20211128T145647>,
 <Item id=S2B_MSIL2A_20211125T132229_R038_T23KLT_20211126T0100

## Process the Images

In [13]:
import gc
from datetime import datetime

cluster_bands = ['mndwi', 'ndwi', 'B12']

lap = datetime.now()

for tile in tiles:
    clear_output(wait=True)
        
    print(datetime.now() - lap)
    lap = datetime.now()
    
    # tr.create_snapshot()
    # tr.stats.print_summary()
    
    print('*'*50)
    print(f'Processing tile {tile}')
    wd = WaterDetect(tile, cluster_bands, s2clouds=True, n_jobs=7)
    wd.run_detect_water()
    
    # save thumbnails
    wd.plot_thumbs(cols=3, thumbs=['rgb', 'watermask', 'mndwi', 'mask', 'glint', 'ndwi'],  save_folder='./tmp')
    
    # save graphs
    wd.plot_graphs([['ndwi', 'mndwi'], ['B12', 'mndwi']], cols=2, save_folder='./tmp')
    
    # save the watermask
    wd.save_geotiff('nodata_watermask', Path('./tmp')/f'{wd.img_item.id[:38]}_watermask.tif')

    # del wd
    # gc.collect()

0:03:20.143768
**************************************************
Processing tile <Item id=S2A_MSIL2A_20210713T132241_R038_T23KLT_20210714T070004>
Getting image: S2A_MSIL2A_20210713T132241_R038_T23KLT_20210714T070004
Retrieving band info
Dividing band in 4 tiles


Tiles:   0%|          | 0/4 [00:00<?, ?tile/s]

Tile[0:2896,0:2896]:   0%|          | 0.00/10.9M [00:00<?, ?b/s]

Tile[0:2896,2896:5490]:   0%|          | 0.00/9.77M [00:00<?, ?b/s]

Tile[2896:5490,2896:5490]:   0%|          | 0.00/8.75M [00:00<?, ?b/s]

Tile[2896:5490,0:2896]:   0%|          | 0.00/9.77M [00:00<?, ?b/s]

Finished. The result can be accessed at obj.array
Getting metadata: granule-metadata


D:\git_projects\pc_waterdetect\waterdetect\glint.py:54: RuntimeWarning: Mean of empty slice
  return np.nanmean(arrays_lst, axis=0)


RasterioIOError: Read or write failed. IReadBlock failed at X offset 6, Y offset 7: TIFFReadEncodedTile() failed.